# Analyse de Stocks avec Modèles LSTM

## Problématique

Ce projet vise à analyser 100 actions américaines du S&P 500 en utilisant des modèles LSTM (Long Short-Term Memory) pour prédire les mouvements de prix et comparer deux approches :

1. **Modèle Global** : Un seul modèle LSTM entraîné sur toutes les actions
2. **Modèles par Secteur** : Un modèle LSTM par secteur (11 modèles au total)

L'objectif est de comparer la performance ajustée au risque entre ces deux approches.



## Recherche et Justifications

### 1. Choix de l'Univers d'Investissement

**100 Actions Américaines du S&P 500**

- **Diversification sectorielle maximale** : 11 secteurs différents représentés (GICS)
- **~9-10 actions par secteur** en moyenne
- **Grandes capitalisations** : Liquidité élevée et données complètes
- **Représentativité du marché** : Couverture d'environ 80% de la capitalisation boursière US

### 2. Architecture des Modèles

**Modèle LSTM avec architecture séquentielle :**
- LSTM(64) → Dropout(0.2) → LSTM(32) → Dropout(0.2) → Dense(32) → Dropout(0.2) → Dense(1)
- Classification binaire : Hausse (1) ou Baisse (0)
- Fenêtre temporelle : 20 jours de données pour prédire le jour suivant

### 3. Méthode : Rolling Window

**Pourquoi Rolling Window plutôt qu'Extending Window ?**

- **Rolling Window** : Fenêtre de taille fixe qui glisse dans le temps (plus réaliste)
- **Extending Window** : Utilise toutes les données jusqu'à un point (peut créer du look-ahead bias)
- **Configuration** : 252 jours d'entraînement (~1 an), 63 jours de test (~3 mois)

### 4. Métriques de Performance

**Métriques calculées :**
- **Précision** : Pourcentage de prédictions correctes
- **Rendement stratégie** : Performance cumulée du modèle
- **Rendement Buy & Hold** : Performance d'achat et conservation
- **Performance relative** : Surperformance vs Buy & Hold

**Métriques ajustées au risque :**
- **Sharpe Ratio** : Rendement ajusté à la volatilité
- **Sortino Ratio** : Rendement ajusté au risque de baisse uniquement
- **Maximum Drawdown** : Perte maximale observée
- **Volatilité annualisée** : Mesure du risque



## Structure du Projet

```
projet_dauphine_python/
├── main.py                 # Point d'entrée principal
├── src/
│   ├── __init__.py
│   ├── fetch_data.py       # Téléchargement des données
│   ├── data_processing.py   # Traitement et préparation des données
│   └── strategy.py         # Modèles LSTM et stratégies
├── outs/                    # Tous les fichiers de sortie
│   ├── stock_analysis.db
│   ├── results_sector.csv
│   ├── results_global.csv
│   └── graphique_*.png
└── notebook/
    └── analyse_stocks_lstm.ipynb  # Ce notebook
```



In [ ]:
# Import des modules
import sys
sys.path.append('..')

from src.fetch_data import STOCKS
import pandas as pd

# Afficher la liste des actions
print(f"Nombre total d'actions : {len(STOCKS)}")
print("\nRépartition par secteur :")
df_stocks = pd.DataFrame(STOCKS)
print(df_stocks['sector'].value_counts().sort_index())



## Hypothèses de Recherche

### Hypothèse 1 : Modèles par Secteur > Modèle Global

**Justification :**
- Les secteurs ont des dynamiques différentes (ex: Technologie vs Utilitaires)
- Un modèle spécialisé par secteur peut mieux capturer les patterns spécifiques
- Moins de bruit dans les données d'entraînement

### Hypothèse 2 : Modèle Global > Modèles par Secteur

**Justification :**
- Plus de données d'entraînement = meilleure généralisation
- Capture les patterns communs à tous les secteurs
- Moins de risque de surapprentissage

### Hypothèse 3 : Performance ajustée au risque

**Objectif :**
- Identifier quel modèle offre le meilleur ratio rendement/risque
- Utiliser le Sharpe Ratio comme métrique principale
- Analyser la volatilité et le maximum drawdown



## Résultats et Analyses

### Exécution de l'analyse

Pour lancer l'analyse complète, exécutez :

```bash
python main.py
```

### Fichiers générés

Tous les fichiers de sortie sont sauvegardés dans le dossier `outs/` :

- **stock_analysis.db** : Base de données SQLite avec tous les résultats
- **results_sector.csv** : Résultats détaillés du modèle par secteur
- **results_global.csv** : Résultats détaillés du modèle global
- **graphique_1_comparaison_performances.png** : Comparaison des performances moyennes
- **graphique_2_distribution_sharpe.png** : Distribution des Sharpe Ratios
- **graphique_3_top10_performances.png** : Top 10 actions par performance
- **graphique_4_performance_vs_risque.png** : Scatter plot Performance vs Sharpe Ratio
- **graphique_5_metriques_risque.png** : Comparaison des métriques ajustées au risque



## 📊 Explication des Graphiques Générés

### Graphique 1 : Comparaison des Performances Moyennes

**Type :** Graphique en barres groupées

**Ce qu'il montre :**
- Compare le Modèle Global vs Modèle par Secteur sur 3 métriques
- **Rendement Stratégie** : Gain moyen du modèle
- **Performance** : Surperformance vs Buy & Hold
- **Précision** : Pourcentage de prédictions correctes

**Comment lire :** Plus les barres sont hautes, mieux c'est. Comparez les barres de même couleur entre les deux modèles.

---

### Graphique 2 : Distribution des Sharpe Ratios

**Type :** Boîte à moustaches (Boxplot)

**Ce qu'il montre :**
- Distribution des Sharpe Ratios pour chaque modèle
- Le **Sharpe Ratio** mesure le rendement ajusté au risque
- Plus c'est élevé, meilleur est le ratio rendement/risque

**Comment lire :**
- Le rectangle central = 50% des valeurs
- La ligne au milieu = médiane
- Les moustaches = valeurs min/max
- Le point = moyenne

**Interprétation :** Un Sharpe Ratio > 0 est bon. Plus il est élevé, mieux c'est.

---

### Graphique 3 : Top 10 Actions par Performance

**Type :** Barres horizontales

**Ce qu'il montre :**
- Les 10 meilleures actions selon le modèle par secteur
- Performance relative : surperformance vs Buy & Hold

**Couleurs :**
- 🟢 Vert = Performance positive (le modèle fait mieux)
- 🔴 Rouge = Performance négative (le modèle fait moins bien)

**Comment lire :** Les actions sont classées du meilleur (en haut) au moins bon. Plus la barre s'étend vers la droite, meilleure est la performance.

---

### Graphique 4 : Performance vs Risque Ajusté

**Type :** Nuage de points (Scatter Plot) - 2 panneaux

**Ce qu'il montre :**
- Relation entre Performance (Y) et Sharpe Ratio (X) pour chaque action
- Chaque point = une action
- Couleur = performance (vert = positif, rouge = négatif)

**Quadrants :**
- **Haut-Droite** 🎯 : MEILLEUR - Bonne performance + bon risque
- **Haut-Gauche** : Bonne performance mais risque élevé
- **Bas-Droite** : Faible performance mais risque maîtrisé
- **Bas-Gauche** ⚠️ : PIRE - Faible performance + risque élevé

**Comment lire :** Cherchez les points en haut à droite = actions idéales !

---

### Graphique 5 : Métriques Ajustées au Risque

**Type :** Barres groupées

**Ce qu'il montre :**
- 4 métriques de risque comparées entre les deux modèles :
  1. **Sharpe Ratio** : Rendement/risque total (plus haut = mieux)
  2. **Sortino Ratio** : Rendement/risque de baisse (plus haut = mieux)
  3. **Max Drawdown** : Perte maximale (plus bas = mieux)
  4. **Volatilité** : Amplitude des variations (plus bas = mieux)

**Comment lire :** Comparez les barres de même couleur. Pour Sharpe/Sortino : plus haut = mieux. Pour Drawdown/Volatilité : plus bas = mieux.

---

> 💡 **Astuce** : Regardez tous les graphiques ensemble pour avoir une vue complète. Le graphique 5 est souvent le plus important car il combine performance ET risque.



In [ ]:
# Visualiser les graphiques générés (si disponibles)
import os
from IPython.display import Image, display

output_dir = '../outs'
graphiques = [
    'graphique_1_comparaison_performances.png',
    'graphique_2_distribution_sharpe.png',
    'graphique_3_top10_performances.png',
    'graphique_4_performance_vs_risque.png',
    'graphique_5_metriques_risque.png'
]

print("📊 Graphiques disponibles :\n")
for i, graphique in enumerate(graphiques, 1):
    path = os.path.join(output_dir, graphique)
    if os.path.exists(path):
        print(f"✓ Graphique {i}: {graphique}")
        try:
            display(Image(path))
        except:
            print(f"  (Impossible d'afficher l'image)")
    else:
        print(f"✗ Graphique {i}: {graphique} (non trouvé)")
    print()

